In [207]:
import pandas as pd
import talib as ta
import numpy as np
import plotly.graph_objs as go

import datetime as dt
from datetime import date
import pandas_datareader.data as pdr
import os

In [208]:
def get_stock_data_Internet(code) :
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [209]:
def get_stock_data_local(dirname, code) :
  dir_path = f"{dirname}"
  csv_name = f'{code}_JP.csv'
  csv_path = os.path.join(dir_path, csv_name)
  dtype_dict = {'Open': float, 'High':float, 'Low':float, 'Close':float, 'Volume':float}
  df = pd.read_csv(csv_path, index_col=0, usecols=["Date", "Open", "High", "Low", "Close", "Volume"],
                  dtype=dtype_dict)
  df.index = pd.to_datetime(df.index).astype('<M8[ns]')
  
  return df

In [210]:
def GCDC(dirname, code, day) :
        gc_flag = False
        dc_flag = False
        #ここを変えればよい
        df = get_stock_data_local(dirname, code)
        close =df["Close"]
        #移動平均線
        df["ma5"] = ta.SMA(close, timeperiod=5)
        df['ma25'] = ta.SMA(close, timeperiod=25)

        #ゴールデンクロス、デッドクロス
        cross = df["ma5"] > df["ma25"]
        df["cross"] = cross
        cross_shift = cross.shift(1)
        temp_gc = (cross != cross_shift) & (cross ==True)
        temp_dc = (cross != cross_shift) & (cross == False)
        gc = [m if g == True else np.nan for g, m in zip(temp_gc, df['ma5'])]
        dc = [m if d == True else np.nan for d, m in zip(temp_dc, df['ma25'])]
        df["gc"], df["dc"] = gc, dc


        #------------------------日付調整-------------------------------
        today = date.today()
        few_ago_days = today - dt.timedelta(days = day)
        rdf = df[dt.datetime(few_ago_days.year, few_ago_days.month, few_ago_days.day):]
        rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

        #GCまたはDCをしている日付の抽出
        gc_index = rdf.query("gc == gc").index
        dc_index = rdf.query("dc == dc").index

        if len(gc_index) != 0 :
                gc_flag = True

        if len(dc_index) != 0 :
                dc_flag = True

        return gc_flag, dc_flag

In [245]:
#GDDCをした銘柄
dirname = "Prime_data"
day = 5

cn = pd.read_csv('all_stock_data.csv')
cn = cn.drop(['33IndustryClass', '33IndustryCode', '17IndustryClass', '17IndustryCode'], axis = 1)
if dirname == "Prime_data" :
    cn = cn[cn['MarketSegment'].isin(['プライム（内国株式）'])]
if dirname == "Standard_data" :
    cn = cn[cn['MarketSegment'].isin(['スタンダード（内国株式）'])]
if dirname == "Growth_data" :
    cn = cn[cn['MarketSegment'].isin(['グロース（内国株式）'])]

codes = cn["Code"]

cn["GC"] = np.nan
cn["DC"] = np.nan

for code in codes :
    target_index = cn.query(f"Code == {code}").index[0]
    print(code, end=' : ')
    if GCDC(dirname, code, day)[0] == True:
        cn.loc[[target_index],"GC"] = True
        print("GC" , end = " ")
    if GCDC(dirname, code, day)[1] == True:
        cn.loc[[target_index],"DC"] = True
        print("DC")
        continue
    else :
        print("None")

1301 : GC None
1332 : None
1333 : None
1375 : None
1377 : None
1379 : None
1414 : GC None
1417 : None
1419 : None
1429 : None
1430 : None
1433 : None
1435 : None
1515 : GC None
1518 : None
1605 : None
1662 : None
1663 : None
1712 : None
1716 : None
1719 : None
1720 : GC None
1721 : GC None
1726 : None
1762 : None
1766 : None
1780 : None
1786 : None
1801 : GC None
1802 : None
1803 : None
1805 : GC None
1808 : None
1810 : GC None
1812 : None
1813 : None
1814 : None
1815 : None
1820 : None
1821 : None
1822 : None
1833 : GC None
1835 : None
1847 : None
1852 : None
1860 : GC None
1861 : None
1870 : GC None
1871 : GC None
1873 : None
1878 : None
1879 : None
1882 : None
1884 : None
1885 : None
1887 : None
1888 : None
1890 : None
1893 : GC None
1898 : None
1899 : None
1911 : None
1925 : None
1926 : None
1928 : GC None
1929 : None
1930 : None
1934 : None
1938 : None
1939 : None
1941 : None
1942 : None
1944 : None
1945 : GC None
1946 : None
1949 : None
1950 : None
1951 : None
1952 : GC None
1954